In [1]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
import missingno as msno
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.stats import norm


In [2]:
# Set the figure size - handy for larger output
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [10, 6]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

# import data

In [4]:
data = pd.read_pickle('D:\MARBURG VIRUS DISEASES\DATA_PREPROCESSING/data_concat_countries_and_wb_groups_income.pkl')

data.head(1)

(932, 122)

In [11]:
country_data = pd.read_pickle('D:\MARBURG VIRUS DISEASES\DATA_PREPROCESSING/countries_split.pkl').drop('Addresses', axis=1)
country_data.head(1)

,Afghanistan,United States,Angola,Argentina,Australia,Austria,Bangladesh,Belgium,Belize,Brazil,...,Thailand,Turkey,United Arab Emirates,Uganda,Ukraine,Venezuela,Vietnam,Zambia,Zimbabwe,Sudan
UT (Unique WOS ID),,,,,,,,,,,,,,,,,,,,,
WOS:000904661800001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data_prep = pd.read_excel(f'D:\MARBURG VIRUS DISEASES/data/WOSCC/woscc_bon_23_02_verif_on_wos.xls')
data_prep = data_prep.rename(columns={"UT (Unique WOS ID)": "wos_ID"})
data_prep = data_prep.applymap(lambda s:s.lower() if type(s) == str else s)
#data_prep = data_prep.drop_duplicates('wos_ID')
data_prep.head(1)

,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,Source Title,...,Web of Science Index,Research Areas,IDS Number,Pubmed Id,Open Access Designations,Highly Cited Status,Hot Paper Status,Date of Export,wos_ID,Web of Science Record
0,j,"bi, jh; wang, hj; han, qx; pei, hy; wang, hl; ...",NaN,NaN,NaN,"bi, jinhao; wang, haojie; han, qiuxue; pei, ho...",NaN,NaN,a rabies virus-vectored vaccine expressing two...,emerging microbes & infections,...,science citation index expanded (sci-expanded),immunology; infectious diseases; microbiology,7j5zq,36453198.0,"green accepted, gold",NaN,NaN,2023-02-22,wos:000904661800001,0


In [26]:
data_prep.columns

Index(['Publication Type', 'Authors', 'Book Authors', 'Book Editors',
       'Book Group Authors', 'Author Full Names', 'Book Author Full Names',
       'Group Authors', 'Article Title', 'Source Title', 'Book Series Title',
       'Book Series Subtitle', 'Language', 'Document Type', 'Conference Title',
       'Conference Date', 'Conference Location', 'Conference Sponsor',
       'Conference Host', 'Author Keywords', 'Keywords Plus', 'Abstract',
       'Addresses', 'Affiliations', 'Reprint Addresses', 'Email Addresses',
       'Researcher Ids', 'ORCIDs', 'Funding Orgs', 'Funding Name Preferred',
       'Funding Text', 'Cited References', 'Cited Reference Count',
       'Times Cited, WoS Core', 'Times Cited, All Databases',
       '180 Day Usage Count', 'Since 2013 Usage Count', 'Publisher',
       'Publisher City', 'Publisher Address', 'ISSN', 'eISSN', 'ISBN',
       'Journal Abbreviation', 'Journal ISO Abbreviation', 'Publication Date',
       'Publication Year', 'Volume', 'Issue', 'Pa

# Extract indicators

## Extract number of countries

In [15]:
country_data['number_of_country'] = country_data.notnull().sum(axis=1)
country_data.head()

,Afghanistan,United States,Angola,Argentina,Australia,Austria,Bangladesh,Belgium,Belize,Brazil,...,Turkey,United Arab Emirates,Uganda,Ukraine,Venezuela,Vietnam,Zambia,Zimbabwe,Sudan,number_of_country
UT (Unique WOS ID),,,,,,,,,,,,,,,,,,,,,
WOS:000904661800001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
WOS:000921279800001,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
WOS:000909001200001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
WOS:000777045900001,NaN,1.0,NaN,NaN,2.0,NaN,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
WOS:000849602200001,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


## Extract number of institution

In [8]:
# number of institutions
data_prep['number_of_institution'] =  pd.DataFrame(data_prep['Addresses'].str.split(';', expand=True)).count(axis='columns')
data_prep['number_of_institution']#.rename(columns={0: "Number_of_author_per_study"})

0      20
1      15
2       1
3      16
4       7
       ..
927     0
928     0
929     0
930     0
931     0
Name: number_of_institution, Length: 932, dtype: int64

## Extract number of authors

In [9]:
# number of authors
data_prep['number_of_author'] =  pd.DataFrame(data_prep['Author Full Names'].str.split(';', expand=True)).count(axis='columns')#.rename(columns={0: "Number_of_author_per_study"})
data_prep['number_of_author']

0      13
1      11
2       1
3      14
4       6
       ..
927     2
928     5
929     3
930     3
931     4
Name: number_of_author, Length: 932, dtype: int64

## Extract types of collaboration

In [16]:
country_data['number_of_country'].value_counts()

2     558
3     197
4      65
1      57
5      28
7       9
6       9
8       4
11      1
12      1
14      1
9       1
10      1
Name: number_of_country, dtype: int64

In [20]:
# Create new column based 
country_data['collab_types'] = country_data['number_of_country'].apply(lambda x: 'Nat. collab.' if x <= 1 else 'Int. collab.')
country_data['collab_types']

UT (Unique WOS ID)
WOS:000904661800001    Int. collab.
WOS:000921279800001    Int. collab.
WOS:000909001200001    Int. collab.
WOS:000777045900001    Int. collab.
WOS:000849602200001    Int. collab.
                           ...     
WOS:A1968B358900049    Nat. collab.
WOS:A1968A855300001    Nat. collab.
WOS:A1968A855300006    Nat. collab.
WOS:A1968A855300008    Nat. collab.
WOS:A1968A855300007    Nat. collab.
Name: collab_types, Length: 932, dtype: object

In [21]:
country_data['collab_types'].value_counts()

Int. collab.    875
Nat. collab.     57
Name: collab_types, dtype: int64

In [24]:
data_region= data[['north_africa', 'eastern_africa', 'Southern_africa', 'Western_africa', 'Central_africa', 'non_african_countries', 
            'wb_east_asia_and_pacific', 'wb_europe_and_central_asia', 'wb_latin_america_and_the_caribbean', 
            'wb_middle_east_and_north_africa', 'wb_north_america', 'wb_south_asia', 'wb_sub_saharan_africa'
             ]]

In [25]:
data_region.notnull().sum(axis=1).value_counts()

2    612
3    142
4     65
0     57
5     38
6     13
7      3
1      1
8      1
dtype: int64